In [1]:
!pip install unidecode



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pandas as pd
import numpy as np
import json
import re
import unidecode

pd.options.display.float_format = '{:.1f}'.format

# Transformando o JSON em DataFrame
with open('dados/lista_carros.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.json_normalize(data)

# Removendo os '.' p
df['price'] = df['price'].apply(lambda x: x.replace('.', '').strip()).astype('float64')


# Adicionando as colunas e preenchendo com False
df['Com IPVA pago'] = df['services'].apply(lambda x: 'Com IPVA pago' in x)
df['Aceita troca'] = df['services'].apply(lambda x: 'Aceita troca' in x)
df['Entrega a domicílio'] = df['services'].apply(lambda x: 'Entrega a domicílio' in x)
df['Único dono'] = df['services'].apply(lambda x: 'Único dono' in x)
df['Com preço negociável'] = df['services'].apply(lambda x: 'Com preço negociável' in x)
df['Higienização completa'] = df['services'].apply(lambda x: 'Higienização completa' in x)
df['Não aceita troca'] = df['services'].apply(lambda x: 'Não aceita troca' in x)
df['Com garantia mecânica'] = df['services'].apply(lambda x: 'Com garantia mecânica' in x)
df['Test drive a domicilio'] = df['services'].apply(lambda x: 'Test drive a domicilio' in x)

# Inserindo as novas colunas e os valores
df['Marca'] = ''
df['Modelo'] = ''
df['Ano'] = ''
df['Versão'] = ''
df['Cor'] = ''
df['Tipo de combustível'] = ''
df['Motor'] = ''
df['Tipo de carroceria'] = ''
df['Quilometros'] = ''

for i, row in df.iterrows():
    response_details = row['response_details']
    if len(response_details) >= 9:
        df.at[i, 'Marca'] = response_details[0]
        df.at[i, 'Modelo'] = response_details[1]
        df.at[i, 'Ano'] = response_details[2]
        df.at[i, 'Versão'] = response_details[3]
        df.at[i, 'Cor'] = response_details[4]
        df.at[i, 'Tipo de combustível'] = response_details[5]
        df.at[i, 'Motor'] = response_details[6]
        df.at[i, 'Tipo de carroceria'] = response_details[7]
        df.at[i, 'Quilometros'] = response_details[8]

# Excluindo as colunas "name", "services" e "details" e "response_details"
df = df.drop(['name', 'services', 'details', 'response_details'], axis=1)

In [20]:
filtro_km = r'^\d+ km$'

# remover valores que não correspondem ao padrão de quilometragem
df['Quilometros'] = df['Quilometros'].apply(lambda x: x if re.match(filtro_km, str(x)) else np.nan)

# converter a coluna 'Quilometros' para float64
df['Quilometros'] = df['Quilometros'].str.replace(' km', '').astype('float64')

# preencher valores vazios com a mediana da coluna 'Quilometros'
mediana = df['Quilometros'].median()
df['Quilometros'] = df['Quilometros'].fillna(mediana)


In [21]:
# Removendo os dados que não pertencem a Cor no dataframe
df = df[~df['Cor'].isin(['OUTRO', 'Gasolina e álcool', 'Diesel'])]


In [22]:
# Removendo os dados que não pertencem a tipo de combustível
df = df[~df['Tipo de combustível'].isin(['4', '5', '2', '3','Híbrido/Gasolina', 'Gasolina e Elétrico (híbrido)', 'FLEX'])]

In [23]:
# Crie uma coluna 'Portas' e transfira os dados da coluna 'Motor' para ela
df['Portas'] = df['Motor']


In [24]:
df = df[~df['Tipo de carroceria'].isin(['1.6', '3.0', '1.0 tgdi', '2.3 bi turbo', 'SUV', 'Hatch' ])]

In [25]:
df['Tipo de carroceria'].unique()

array(['Automática', 'Manual', '', 'Semiautomática',
       'Automática sequencial', 'CÂMBIO AUTOMÁTICO', 'AUTOMÁTICO'],
      dtype=object)

In [26]:
df['Tipo de carroceria'] = df['Tipo de carroceria'].apply(lambda x: x.replace('CÂMBIO AUTOMÁTICO', 'Automática').strip())
df['Tipo de carroceria'] = df['Tipo de carroceria'].apply(lambda x: x.replace('AUTOMÁTICO', 'Automática').strip())

In [27]:
df = df.rename(columns={'Tipo de carroceria': 'Tipo de transmissão'})


In [28]:
# Limpar a coluna df['Motor']
df['Motor'] = df['Motor'].str.strip()

# Extrair valores numéricos da coluna df['Tipo de carroceria'] e armazená-los na coluna df['Motor']
df['Motor'] = df['Versão'].str.extract(r'([\d\.]+)').fillna('').astype(str)


In [29]:
df = df.drop("Versão", axis=1)

In [30]:
# pd.options.display.float_format = '{:.2f}'.format
df['Motor'] = pd.to_numeric(df['Motor'], errors='coerce')
df['Motor'] = df['Motor'][~np.isnan(df['Motor'])]
df['Motor'] = [format(x, '.1f') for x in df['Motor'].astype(float)]


In [31]:
df = df[~df['Motor'].isin(['1500.0', '200.0','208.0','2500.0', '35150.0', '35.0', '0.0', '2021.0',
       '417.0','430.0', '180.0', '75.0', '12.0', '270.0','16.0', '2018.0',
       '250.0', '320.0','307.0', 'nan'])]



In [42]:
df['modelo'] = df['modelo'].apply(lambda x: x.replace('GOL MC4 _4P_', 'Gol MC4').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('TORO RANCH 2.0 16V 4X4 DIESEL ATU.', 'Toro Ranch').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('TRACKER PREMIER 1.2 TURBO 12V FLEX AUT.', 'Tracker Premier').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('TRACKER PREMIER 1.2 TURBO 12V FLEX AUT', 'Tracker Premier').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('XE 2.0 TURBOCHARGED PURE 240CV AUT', 'XE').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('LOGAN EXPRESSION SCE _4P_', 'Logan Expression').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('PALIO 1.0 ECONOMY FIRE FLEX 8V 4P', 'Palio').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('TORO RANCH 2.0 16V 4X4 DIESEL AUT.', 'Toro Ranch').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('CR-V LX 2.0 AUT', 'CR-V LX').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('SIENA ESSENCE 1.6 16V FLEX MEC.', 'Siena Essence').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('COROLLA CROSS XRE 2.0', 'Corolla Cross').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('LIVINA XGEAR 1.8 AUT', 'Livina XGEAR').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('HR-V EX 1.8 FLEXONE 16V 5P AUT', 'HR-V EX').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('DUSTER 1.6 EXPRESSION', 'DUSTER EXPRESSION').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('ECOSPORT TIT AT 2.0', 'Ecosport TIT').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('FUSION 2.0 TITANIUM AWD 16V GASOLINA 4P AUTOMÁTICO', 'Fusion').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('SENTRA SV 2.0 FLEX 16V AUT.', 'Sentra SV').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('CRETA 1.6 16V FLEX ATTITUDE AUT', 'Creta').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('GRAND SIENA 1.0 EVO FLEX ATTRACTIVE MANUAL', 'Grand Siena').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('WRANGLER SPT 3.6L', 'WRANGLER SPT').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('ONIX 1.4MT LTZ', 'Onix').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('TIIDA 1.8 SL', 'TIIDA').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('VERSA 1.6 16V FLEX SL 4P MANUAL', 'Versa').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('FOX 1.0 8V (G2) 4P', 'Fox').strip())
df['modelo'] = df['modelo'].apply(lambda x: x.replace('LOGAN EXPRESSION SCE _4P_P', 'Logan Expression').strip())

In [43]:
df.columns = [unidecode.unidecode(col).lower().replace(' ', '_') for col in df.columns]



In [44]:
df

,price,com_ipva_pago,aceita_troca,entrega_a_domicilio,unico_dono,com_preco_negociavel,higienizacao_completa,nao_aceita_troca,com_garantia_mecanica,test_drive_a_domicilio,marca,modelo,ano,cor,tipo_de_combustivel,motor,tipo_de_transmissao,quilometros,portas
0,156990.0,True,True,False,False,False,False,False,False,False,Hyundai,Tucson,2020,Branco,Gasolina,1.6,Automática,35000,4
1,85990.0,False,True,False,False,False,True,False,False,False,Chevrolet,Onix,2023,Cinza-escuro,Gasolina e álcool,1.0,Manual,65000,4
2,53990.0,False,True,False,False,False,False,False,False,False,Ford,Ka,2018,Branco,Gasolina e álcool,1.0,Manual,65000,4
4,85990.0,False,True,False,False,False,True,False,False,False,Jeep,Renegade,2018,Cinza-escuro,Gasolina e álcool,1.8,Automática,65000,4
5,103890.0,False,True,False,False,False,False,False,False,False,Jeep,Compass,2017,Prateado,Gasolina e álcool,2.0,Automática,65000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009,66990.0,True,True,False,False,False,True,False,False,False,Nissan,Versa,2018,Preto,Gasolina e álcool,1.6,Automática,65000,4
2011,48800.0,True,True,False,False,False,True,False,False,False,Toyota,Corolla,2011,Prateado,Gasolina-Álcool e gás natural,2.0,Automática,65000,4
2012,54800.0,True,True,False,False,False,True,False,False,False,Volkswagen,Voyage,2019,Prateado,Gasolina e álcool,1.6,Manual,65000,4
2013,79900.0,False,False,False,False,False,False,False,False,False,Honda,WR-V,2018,Cinza-escuro,Gasolina e álcool,1.5,Automática,65000,5


In [36]:
(df['quilometros']).unique()

array([ 35000.,  65000., 150000.,  49000.,  11590., 143000.,  24200.,
        44700., 168000., 113653.,  11000., 145000.,      0., 138594.,
       118500.,  84793.,  50800., 109376.,  48609.,  55998., 144000.,
       110340.,  71000.,  32590., 118764.,  80045.,  26000.,  59734.,
        76495.,  38474., 101000.,  27500.,  16000.,  91000.,  11775.,
        30000.,  14578.,  16535.,  90300.,  15000.,  55415.,  48000.,
        70000.,  62000.,   2300.,  46000.,  90000.,  31549.,  93000.,
       108000.,  34770., 127000.,  70326., 110351., 149000.,  49884.,
        38000.,  51349.,  50084.,  92000., 129000.,  77136.,  52270.,
        70342.,  28000.,  49658., 166995., 185359.,  56000.,  78219.,
        60000.])

In [37]:
df['motor'] = df['motor'].astype('object')

In [38]:
df['portas'] = df['portas'].astype('int64')

In [39]:
df['quilometros'] = df['quilometros'].astype('int64')

In [40]:
df.dtypes

price                     float64
com_ipva_pago                bool
aceita_troca                 bool
entrega_a_domicilio          bool
unico_dono                   bool
com_preco_negociavel         bool
higienizacao_completa        bool
nao_aceita_troca             bool
com_garantia_mecanica        bool
test_drive_a_domicilio       bool
marca                      object
modelo                     object
ano                        object
cor                        object
tipo_de_combustivel        object
motor                      object
tipo_de_transmissao        object
quilometros                 int64
portas                      int64
dtype: object

In [23]:
df.to_csv('df.csv',index='False')

In [24]:
df = pd.read_csv('df.csv')

In [25]:
df = pd.DataFrame(df)

In [26]:
df.rename(columns={'Unnamed: 0': 'carros_id'}, inplace=True)


In [45]:
df['modelo'].unique()

array(['Tucson', 'Onix', 'Ka', 'Renegade', 'Compass', 'HB20', 'Corolla',
       'Strada', 'Master', 'Toro', 'Up!', 'Voyage', 'Uno', 'Mobi', 'Idea',
       'Celta', 'S-10', 'HB20S', 'Sandero', 'Spacefox', 'Palio Adventure',
       'Argo', 'Yaris', 'Golf', 'Fox', 'Gol', 'Cross Up', 'Polo',
       'Tiguan', 'New Beetle', 'Crossfox', 'Fusca', 'Santana', 'HR',
       'Fiesta', 'Polo Hatch', 'Jetta', 'Virtus', 'Nivus', 'Taos',
       'T-Cross', 'Express', 'Amarok', 'Sportage', 'RAV4', 'Sprinter Van',
       'Range Rover Evoque', 'Cayenne', 'AmarokMLB1744', 'Corolla Cross',
       'Creta', 'Saveiro', 'Gol MC4', 'Kombi', 'Polo Sedan',
       'Outros Modelos', 'Kombi FurgãO', 'Fiorino', 'Daily Chasis',
       'Civic', 'Captur', 'Toro Ranch', 'Q5', 'Ducato', 'Punto', 'SW4',
       'Daily', 'Sprinter Vidrado', 'IX35', 'Focus', 'Zafira', 'Palio',
       'MASTER FURGAO L1H1', 'Master FurgãO', 'Tracker', 'ASX', 'GLA 250',
       'Q3', 'Duster', 'XC60', 'Cronos', 'I30', 'Stilo', 'S10', 'C 180',
     